# lite

In [ ]:
import json
import os
import re
from pathlib import Path

import importnb

with importnb.Notebook():
    from jupyak.tasks import _actions as A
    from jupyak.tasks import _py as P
    from jupyak.tasks import _well_known as W
    from jupyak.tasks import _yak as Y

In [ ]:
def lite_tasks(yak: Y.Yak):
    work_path = yak.work_path
    lite = yak.lite.work_path
    build_conf = yak.lite.build_config_path
    in_work_lite = {"cwd": lite}
    lite_args = [*yak.env.run_args, "jupyter", "lite"]

    all_deps = cli_deps, conf_deps, conf_targets, build_deps = _find_lite_deps(yak)

    yield dict(
        name="config",
        doc="> configure jupyterlite",
        file_dep=[*conf_deps, *build_deps],
        actions=[(_make_config, [yak, *all_deps])],
        targets=sorted(set(conf_targets)),
    )

    yield dict(
        name="build",
        doc="> build lite site archive",
        file_dep=[*conf_targets, *build_deps, *cli_deps],
        actions=[
            A.run([*lite_args, "build"], in_work_lite),
            A.run([*lite_args, "archive"], in_work_lite),
        ],
        targets=[yak.lite.app_shasums_path],
    )

In [ ]:
def _find_lite_deps(yak: Y.Yak):
    all_deps = cli_deps, conf_deps, conf_targets, build_deps = [], [], [], []
    conf_targets += [yak.lite.build_config_path]
    cli_deps += [yak.env.lab_share / "static" / W.PACKAGE_JSON]

    for name, repo in yak.py_repos.items():
        if repo.lite:
            _add_repo_deps(repo, *all_deps)

    if yak.lite.gist:
        _add_gist_deps(yak, *all_deps)

    self_dist = os.environ.get(W.ENV_VAR_SELF_DIST)

    if self_dist:
        build_deps += [Path(self_dist)]

    return all_deps

In [ ]:
def _add_gist_deps(yak: Y.Yak, cli_deps, conf_deps, conf_targets, build_deps):
    gist_path = yak.lite.gist_path
    work_path = yak.lite.work_path
    conf_deps += [gist_path / ".git/refs/heads" / W.WORK_BRANCH]
    if yak.lite.gist_path.exists():
        gist_files = [p for p in gist_path.glob("*") if not p.is_dir()]
        conf_deps += gist_files
        conf_targets += [work_path / p.name for p in gist_files]

In [ ]:
def _add_repo_deps(repo: Y.Repo, cli_deps, conf_deps, conf_targets, build_deps):
    cli_deps += [
        repo.parent.env.py_site_packages / P._pth_path(pth)
        for pth in repo.lite.needs_pth
    ]
    for ext_js_path, ext in repo.py.lab_extensions.items():
        build_deps += [repo.work_path / t for t in ext.targets]
    if repo.lite.wheel:
        for ppt in repo.py.pyproject_tomls:
            ppt_path = repo.work_path / ppt
            if any(
                re.match(swp, ppt_path.parent.name)
                for swp in repo.lite.skip_wheel_patterns
            ):
                continue
            build_deps += [ppt_path.parent / "dist" / W.SHA256SUMS]

In [ ]:
def _make_config(yak: Y.Yak, cli_deps, conf_deps, conf_targets, build_deps):
    work_path = yak.lite.work_path
    if yak.lite.gist_path.exists():
        A.copy(yak.lite.gist_path, work_path)
    else:
        work_path.mkdir(exist_ok=True, parents=True)
    _patch_build_config(yak, cli_deps, conf_deps, conf_targets, build_deps)

In [ ]:
def _patch_build_config(yak: Y.Yak, cli_deps, conf_deps, conf_targets, build_deps):
    work_path = yak.lite.work_path
    build_config_path = yak.lite.build_config_path
    build_config = {}
    if build_config_path.exists():
        build_config = json.loads(build_config_path.read_text(encoding="utf-8"))

    lbc = build_config.setdefault("LiteBuildConfig", {})
    pla = build_config.setdefault("PipliteAddon", {})

    pkg_jsons = [
        p
        for p in build_deps
        if p.name == W.PACKAGE_JSON
        and (p.parent / "static" / W.THIRD_PARTY_LICENSES).exists()
    ]
    whl_deps = [
        p
        for p in build_deps
        if p.name == W.SHA256SUMS and p.parent.glob(f"*{W.NOARCH_WHL}")
    ]

    rel_output = os.path.relpath(yak.lite.app_path, work_path)
    archive_name = last_known_good_urls()[0]

    lbc.update(
        cache_dir="../.cache/lite",
        contents=["."],
        output_dir=rel_output,
        output_archive=f"{rel_output}/{archive_name}",
        federated_extensions=sorted(
            [
                *lbc.get("federated_extensions", []),
                *[
                    os.path.relpath(pkg_json.parent, work_path)
                    for pkg_json in pkg_jsons
                ],
            ]
        ),
    )

    pla.update(
        piplite_urls=sorted(
            [
                *pla.get("piplite_urls", []),
                *[os.path.relpath(whl_shas.parent, work_path) for whl_shas in whl_deps],
            ]
        ),
    )

    build_config_path.write_text(
        json.dumps(build_config, indent=2, sort_keys=True),
        encoding="utf-8",
    )

In [ ]:
def last_known_good_urls() -> tuple[str, list[str]]:
    pr_or_branch = "unknown"
    url_base = None
    urls = []
    if W.RTD_VERSION:
        pr_or_branch = W.RTD_VERSION
        url_base = f"https://jupyak--{pr_or_branch}.org.readthedocs.build/{W.RTD_LOCALE}/{pr_or_branch}"
    elif W.GH_REF and W.GH_REF.endswith(W.MAIN):
        pr_or_branch = W.MAIN
        url_base = "https://deathbeds.github.io/jupyak"

    archive_name = f"jupyak-{pr_or_branch}-jupyterlite.tgz"

    if url_base:
        static = f"{url_base}/_static"
        urls += [
            f"{static}/{stem}/{archive_name}"
            for stem in ["work/lite", "last-known-good"]
        ]

    return archive_name, urls